<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/5_Dinamic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

# Feature engineering and scaling

In [4]:
data = mf.feature_engineering(data)
data = mf.feature_scaling(data)
data = pd.get_dummies(data)

# Fraud distribution analysis

In [5]:
data

,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D10,D15,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,...,screen_h_732,screen_h_740,screen_h_750,screen_h_756,screen_h_759,screen_h_767,screen_h_768,screen_h_774,screen_h_792,screen_h_800,screen_h_801,screen_h_803,screen_h_809,screen_h_810,screen_h_819,screen_h_837,screen_h_840,screen_h_844,screen_h_845,screen_h_857,screen_h_864,screen_h_880,screen_h_889,screen_h_899,screen_h_900,screen_h_901,screen_h_922,screen_h_924,screen_h_941,screen_h_944,screen_h_945,screen_h_946,screen_h_947,screen_h_960,screen_h_971,screen_h_972,screen_h_973,screen_h_975,screen_h_990,screen_h_992
0,0.0,0.000000e+00,0.002137,0.743044,0.430701,0.381679,0.428948,0.341741,0.877533,0.000213,0.000176,0.0,0.0,0.000000,0.000444,0.0,0.0000,0.004762,0.000000,0.000627,0.000000,0.000343,0.0007,0.048762,0.293226,0.526455,0.517167,0.398642,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,6.359409e-08,0.000900,0.100885,0.536941,0.381679,0.352548,0.347872,0.877533,0.000213,0.000176,0.0,0.0,0.000000,0.000444,0.0,0.0000,0.000000,0.000000,0.000314,0.000000,0.000343,0.0007,0.027489,0.287333,0.526455,0.409871,0.301649,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,4.387992e-06,0.001840,0.210566,0.688839,0.381679,0.474788,0.350937,0.877533,0.000213,0.000176,0.0,0.0,0.000000,0.000444,0.0,0.0000,0.004762,0.000000,0.000314,0.000000,0.000343,0.0007,0.027489,0.287333,0.615201,0.517167,0.398642,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,6.295815e-06,0.001558,0.984824,0.824841,0.381679,0.381198,0.440450,0.877533,0.000427,0.000879,0.0,0.0,0.000000,0.001775,0.0,0.0000,0.004762,0.000000,0.000314,0.000000,0.008568,0.0007,0.197678,0.325411,0.557727,0.517167,0.398642,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,6.740974e-06,0.001558,0.201023,0.731229,0.381679,0.352548,0.406116,0.877533,0.000213,0.000176,0.0,0.0,0.000000,0.000444,0.0,0.0003,0.000000,0.000307,0.000314,0.000000,0.000343,0.0007,0.027489,0.342650,0.576981,0.457082,0.347236,0.956897,0.695902,0.106396,0.181011,0.067046,0.388224,0.166667,0.148396,0.219537,0.097892,0.143266,0.418779,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590528,0.0,9.999947e-01,0.001526,0.319039,0.457283,0.381679,0.589388,0.315378,0.877533,0.000427,0.000176,0.0,0.0,0.002865,0.000000,0.0,0.0000,0.009524,0.000000,0.000314,0.000000,0.001028,0.0014,0.071556,0.312718,0.542232,0.624464,0.495635,0.956897,0.694597,0.103407,0.167129,0.051233,0.301397,0.128920,0.114973,0.194361,0.096386,0.140401,0.436620,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
590529,0.0,9.999948e-01,0.001229,0.542883,0.220782,0.381679,0.585568,0.273687,0.877533,0.000213,0.000176,0.0,0.0,0.000000,0.000444,0.0,0.0000,0.004762,0.000000,0.000314,0.000000,0.000343,0.0007,0.027489,0.287333,0.526455,0.517167,0.398642,0.956897,0.694597,0.103407,0.167129,0.051233,0.401198,0.186992,0.114973,0.194361,0.096386,0.140401,0.436620,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [7]:
initial_splits = 4
splits = 2 
tot_splits = 10   

def split_data(data, tot_splits, verbose = False):
  n_data = {}
  dati_splitted = []
  dati_fraud = []

  for i in range(tot_splits):
    dati_splitted.append(data[(data['TransactionDT']>=(i/tot_splits)) & (data['TransactionDT']<((i + 1)/tot_splits))]) 
    dati_fraud.append(dati_splitted[i][dati_splitted[i]['isFraud'] == 1])
    if verbose: 
      dati_per_split = dati_splitted[i].count()[0]  
      fraud_per_split = dati_fraud[i].count()[0]  
      density = fraud_per_split / dati_per_split
      n_data[i] = [dati_per_split, fraud_per_split, density.round(3)]
  if verbose:
    print(n_data)
    print(dati_splitted)
    print(dati_fraud)

  return dati_splitted, dati_fraud

dati_splitted, dati_fraud = split_data(data, tot_splits, verbose = False)

In [ ]:
dati_fraud

In [ ]:
def train_model(data):
  import xgboost 
  from xgboost import XGBClassifier
  from sklearn.metrics import roc_auc_score

  X_train, X_val, y_train, y_val = mf.split(data, test_size = 0.2)

  clf = XGBClassifier(n_estimators = 100, max_depth = 2, 
                      objective='binary:logistic', learning_rate = 0.5, 
                      tree_method='gpu_hist') #, sample_weight = [1,30] ) 
  clf.fit(X_train.to_numpy(), y_train.to_numpy())

  score = roc_auc_score(y_val, clf.predict_proba(X_val)[:, 1])
  print('ROC AUC score: ', score)
  return clf, score

In [ ]:
def create_ensemble(model_t):
  from sklearn.ensemble import VotingClassifier
  estimators = []
  for model in model_t:
    estimators.append((str(model), model))
    weights.append(1)
  ensemble = VotingClassifier(estimators=estimators, 
                              voting='soft', weights=weights)
  return ensemble

In [ ]:
def dinamic_model(initial_splits, splits, tot_splits, data, data_fraud):
  tmax = np.ceil((tot_splits) / splits)
  data_t = {}
  data_fraud_t = {}
  model_t = {}
  score = {}
  ensemble_score = {}
  print(tmax)
  for t in range(int(tmax - 1)):
    if t == 0:
      start = 0
      end = initial_splits
    else:
      start = splits * (t - 1) + initial_splits
      end = start + splits
      start_fraud = splits * (t - 2) + initial_splits
      end_fraud = start + splits
      data_fraud_t[t] = pd.concat(data_fraud[start_fraud:end_fraud])
    data_t[t] = pd.concat(dati_splitted[start:end])
    if t > 0:
      data_t = pd.concat([data_t[t], data_fraud[t-1]]) #extract_fraud(data, data_t)
    model_t[t], score[t] = train_and_test_model(data_t[t])
    ensemble_t[t] = create_ensemble(model_t)
    if t > 0:
      ensemble_score[t] = get_score(ensemble_t[t])
    if ensemble_score[t] < ensemble_score[t-1]:
      ensemble_t[t] = ensemble_t[t-1]
  return ensemble_t

dinamic_model(initial_splits, splits, tot_splits, data, dati_fraud)

In [7]:
X_train, X_val, y_train, y_val = mf.split(pd.concat(dati_splitted[:initial_splits - 1]), test_size = 0.2)
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
from sklearn.metrics import plot_roc_curve
import xgboost as xgb
from xgboost import XGBClassifier
load = False
if not load:
  clf = XGBClassifier(n_estimators = 4000, max_depth = 10, 
                      objective='binary:logistic', learning_rate = 0.5, 
                      tree_method='gpu_hist') #, sample_weight = [1,30] ) 
  
  clf.fit(X_train.to_numpy(), y_train.to_numpy())

plot_roc_curve(clf, X_val.to_numpy(), y_val.to_numpy())
plt.show()